# 1. Import the required libraries

In [1]:
# Standard code libraries
import os
import platform
import glob

import numpy as np
from numpy.matlib import repmat
from scipy import signal

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Custom code libraries from ReSurfEMG
from resurfemg.config.config import Config
from resurfemg.data_connector import file_discovery
from resurfemg.pipelines import ipy_widgets
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.data_classes.data_classes import (
VentilatorDataGroup, EmgDataGroup)

%matplotlib widget

## 2. Load the ventilator and sEMG data

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

# First find the patients
config = Config()

# Then find the files for the selected patients:
base_path = config.get_directory('test_data')
folder_levels = None

emg_files = file_discovery.find_files(
    base_path=base_path,
    file_name_regex='emg_data_synth_pocc',
    extension_regex='poly5',
    folder_levels=None)

folder_levels = ['date', 'measurement']
vent_files = file_discovery.find_files(
    base_path=base_path,
    file_name_regex='vent_data_synth_pocc',
    extension_regex='poly5',
    folder_levels=None)

# button_list = ipy_widgets.file_select(
#     emg_files,
#     folder_levels=['files'],
#     default_value_select=None,
#     default_idx_select=[0])

In [ ]:
emg_file_chosen = os.path.join(base_path, emg_files['files'].values[0])
vent_file_chosen = os.path.join(base_path, vent_files['files'].values[0])

# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples[:, 2*fs_emg:61*fs_emg]
y_vent = data_vent_samples[:, 2*fs_vent:61*fs_vent]

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/fs_vent for i in range(len(y_vent[0, :]))]

sos = signal.butter(1, 100, 'low', fs=fs_emg, output='sos')
y_emg = 0.75*signal.sosfilt(sos, y_emg)

In [ ]:
# Store the EMG data in a group of TimeSeries objects
emg_timeseries = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi'],
    units=2*['uV'])

# Store the ventilator data in a group of TimeSeries objects
vent_timeseries = VentilatorDataGroup(
    y_vent,
    fs=fs_vent,
    labels=['Paw', 'F', 'Vvent'],
    units=['cmH2O', 'L/s', 'L'])


- Bandpass hp_cf (2-80) lc 50 - 1000
- ECG elimination:
    - Gating: Filling methods
    - ICA: In vs out
    - Wavelet denoising
    - Template subtraction
- Envelope
    - RMS: Window 50 - 400 ms
    - ARV: Window 50 - 400 ms
- Baseline crossing detection
    - Graßhoff: Baseline percentile (20, 33, 40)
    - CoV baseline (Warnaar 2024)
    - Slopesum baseline
    - Slope extrapolation
- Features:
    - ETP
    - Max to baseline
    - Risetime

# 3. Band-pass filtering

In [13]:
x_lim = 9.0
y_lim = 3.0

In [ ]:
# Iterate over filtering order

# Side plot
fig = plt.figure(layout="constrained", figsize=(8, 6))

gs = GridSpec(3, 2, figure=fig)
axes_emg = fig.add_subplot(gs[:, 1])
for i in range(3):
    ax = fig.add_subplot(gs[i, 0], sharex=axes_emg)
axis = np.array(fig.axes)
axes_vent = axis[-3:]


# EMG data
# - Bandpass lc (2-80) hc 50 - 1000
orders = [1, 2, 3, 4, 8]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (order, color) in enumerate(zip(orders, colors)):
    emg_timeseries.filter(order=order, channel_idxs=[1],)
    emg_timeseries.gating(
        channel_idxs=[1],
        gate_width_samples=fs_emg//5
    )
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
# bbox_to_anchor=(0., -0.6, 1., -.6),
axes_emg.legend(
    [str(order) for order in orders], 
    bbox_to_anchor=(0., 1.05, 1., 1.05),
    loc='lower left',
    ncols=5, 
    mode="expand",
    borderaxespad=0.)
axes_emg.set_xlabel('t (s)')
axes_emg.set_ylim([0, y_lim])

# Ventilator data data
# axes_vent = axis[:3]
vent_timeseries.plot_full(axes_vent)
axes_vent[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, x_lim])

In [ ]:
# Iterate over low- and high-pass cut-off frequency
fig = plt.figure(layout="constrained", figsize=(8, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

# EMG data
# - Bandpass lc (2-80) hc 50 - 1000
lp_cfs = [50, 100, 250, 500, 1000]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (lp_cf, color) in enumerate(zip(lp_cfs, colors)):
    # emg_timeseries.filter(lp_cf=lp_cf, channel_idxs=[1], order=1)
    emg_timeseries.filter(lp_cf=lp_cf, channel_idxs=[1])
    emg_timeseries.gating(
        channel_idxs=[1],
        gate_width_samples=fs_emg//5
    )
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_1,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_1.set_xlabel('t (s)')
axes_emg_1.set_ylim([0, y_lim])
axes_emg_1.set_title('Low pass cut-off effects')
axes_emg_1.legend(
    [str(lp_cf) for lp_cf in lp_cfs],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)


# - Bandpass lc (2-80) hc 50 - 1000
hp_cfs = [15, 25, 50, 75, 100]
# hp_cfs = [2]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# colors = ['tab:purple', 'tab:blue', 'tab:cyan', 'tab:olive', 'tab:green',
#           'tab:red', 'tab:orange']

for _, (hp_cf, color) in enumerate(zip(hp_cfs, colors)):
    # emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1], order=1)
    emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1])
    emg_timeseries.gating(
        channel_idxs=[1],
        gate_width_samples=fs_emg//5
    )
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_2,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_2.set_xlabel('t (s)')
axes_emg_2.set_ylim([0, y_lim])
axes_emg_2.set_title('High pass cut-off effects')
axes_emg_2.legend(
    [str(hp_cf) for hp_cf in hp_cfs],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, x_lim])

In [16]:
# Filter
emg_timeseries.filter(
    lp_cf=500,
    hp_cf=25,
    order=3,
)

# 4. ECG-removal

In [ ]:
# Side plot
fig = plt.figure(layout="constrained", figsize=(8, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

# EMG data
# Iterate over window length
gate_width_t_ms = [50, 100, 200, 300, 400]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
# gate_width_t_ms = [400]
# colors = ['tab:orange']
for _, (gate_width_t, color) in enumerate(zip(gate_width_t_ms, colors)):
    emg_timeseries.filter(
        lp_cf=500,
        hp_cf=25,
        order=3,
    )
    gate_width_t_s = (gate_width_t *  fs_emg) // 1000
    emg_timeseries.gating(
        channel_idxs=[1], 
        gate_width_samples=gate_width_t_s,
        fill_method=3)
    emg_timeseries.envelope(channel_idxs=[1])
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_1,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_1.set_xlabel('t (s)')
axes_emg_1.set_ylim([0, y_lim])
axes_emg_1.set_title('Gate-width effects')
axes_emg_1.legend(
    [str(gate_t) for gate_t in gate_width_t_ms],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

peak_idxs = emg_timeseries.channels[1].peaks['ecg'].peak_df['peak_idx'].values
peak_idxs = peak_idxs[peak_idxs < 8 * fs_emg] - 10
t_emg_8 = np.array(t_emg)[np.array(t_emg) < 8 * fs_emg]

t_to_peak = (np.min(np.abs(
    repmat(peak_idxs/fs_emg, len(t_emg_8), 1) 
    - repmat(t_emg_8, len(peak_idxs), 1).T), axis=1))
axes_emg_1.fill_between(t_emg_8, 0.0, y_lim, where=t_to_peak < 1 / (2.5 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, y_lim, where=t_to_peak < 1*3 / (10 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, y_lim, where=t_to_peak < 1 / (5 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, y_lim, where=t_to_peak < 1 / (10 * 2),
                color='black', alpha=0.1)
axes_emg_1.fill_between(t_emg_8, 0.0, y_lim, where=t_to_peak < 1 / (20 * 2),
                color='black', alpha=0.1)


# Iterate over gating fill methods
methods = [0, 1, 2, 3]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red']
for _, (method, color) in enumerate(zip(methods, colors)):
    emg_timeseries.filter(
        lp_cf=500,
        hp_cf=25,
        order=3,
    )
    # emg_timeseries.filter(hp_cf=hp_cf, channel_idxs=[1], order=1)
    emg_timeseries.gating(
        channel_idxs=[1],
        fill_method=method,
        gate_width_samples=fs_emg//5
    )
    emg_timeseries.envelope(channel_idxs=[1], env_window=(fs_emg//5)*2)
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_2,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_2.set_xlabel('t (s)')
axes_emg_2.set_ylim([0, y_lim])
axes_emg_2.set_title('Fill methods')
methods_legend = ['Zero', 'Raw interp', 'Prior av', 'RMS interp']
axes_emg_2.legend(
    methods_legend,
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

axes_emg_2.fill_between(t_emg_8, 0.0, y_lim, where=t_to_peak < 1 / (5 * 2),
                color='black', alpha=0.2)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, x_lim])

# 5. Envelope

In [ ]:
# Gate the EMG
emg_timeseries.filter(
        lp_cf=500,
        hp_cf=25,
        order=3,
    )
emg_timeseries.gating(
    gate_width_samples=fs_emg//5
)


In [ ]:
# Compare envelope methods

# Side plot
fig = plt.figure(layout="constrained", figsize=(8, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

# EMG data
# RMS windows
rms_windows = [100, 200, 250, 500, 1000]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
for _, (rms_window, color) in enumerate(zip(rms_windows, colors)):
    emg_timeseries.envelope(
        channel_idxs=[1], 
        env_type='rms', 
        env_window=(rms_window * fs_emg) // 1000)
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_1,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_1.set_xlabel('t (s)')
axes_emg_1.set_ylim([0, y_lim])
axes_emg_1.set_title('RMS envelope, window length effect')
axes_emg_1.legend(
    [str(rms_window) for rms_window in rms_windows],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)


# ARV windows
arv_windows = [100, 200, 250, 500, 1000]
colors = ['tab:purple', 'tab:blue', 'tab:green', 'tab:red', 'tab:orange']
for _, (arv_window, color) in enumerate(zip(arv_windows, colors)):
    emg_timeseries.envelope(
        channel_idxs=[1], 
        env_type='arv', 
        env_window=(arv_window * fs_emg) // 1000)
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_2,
        signal_type='env',
        baseline_bool=False,
        colors=[color])
axes_emg_2.set_xlabel('t (s)')
axes_emg_2.set_ylim([0, y_lim])
axes_emg_2.set_title('ARV envelope, window length effect')
axes_emg_2.legend(
    [str(arv_window) for arv_window in arv_windows],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, x_lim])

In [20]:
# Calculate the envelope of the signal
emg_timeseries.envelope()

In [ ]:
# Compare baseline methods - On- and offset detection

# Side plot
fig = plt.figure(layout="constrained", figsize=(8, 6))

gs = GridSpec(6, 2, figure=fig)
axes_emg_1 = fig.add_subplot(gs[:3, 1])
axes_emg_2 = fig.add_subplot(gs[3:, 1], sharex=axes_emg_1)
for i in range(3):
    ax = fig.add_subplot(gs[i*2:i*2+2, 0], sharex=axes_emg_1)
axis = np.array(fig.axes)
axes_vent = axis[-3:]

N_plots = 5

# EMG data
emg_di = emg_timeseries.channels[1]
# Moving baseline - windows
bl_windows = [1, 2.5, 5, 7.5, 10]
colors = ['tab:purple', 'tab:cyan', 'tab:green', 'tab:red', 'tab:orange']
for _, (bl_window, color) in enumerate(zip(bl_windows, colors)):
    emg_timeseries.baseline(
        percentile=33,
        window_s=int(bl_window * fs_emg))
    emg_di.detect_emg_breaths(
        peak_set_name='breaths',
        end_idx = int(x_lim * fs_emg),
        prominence_factor = 1.5)
    emg_di.peaks['breaths'].detect_on_offset(
        baseline=emg_di.y_baseline
    )
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_1,
        signal_type='env',
        baseline_bool=True,
        colors=['tab:blue', color])
    emg_di.plot_markers(peak_set_name='breaths', axes=axes_emg_1, colors=color)
axes_emg_1.set_xlabel('t (s)')
axes_emg_1.set_ylim([0.5, y_lim])
axes_emg_1.set_title('Moving baseline - Windows')
axes_emg_1.legend(
    [axes_emg_1.lines[i*N_plots+1] for i in range(len(bl_windows))],
    [str(bl_window) for bl_window in bl_windows],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

for line in [axes_emg_1.lines[i*N_plots] for i in range(len(bl_windows))]:
    line.set_zorder(0)

for line in [axes_emg_1.lines[i*N_plots+1] for i in range(len(bl_windows))]:
    line.set_zorder(5)

for line in [axes_emg_1.lines[i*N_plots+2] for i in range(len(bl_windows))]:
    line.set_zorder(10)

for line in [axes_emg_1.lines[i*N_plots+3] for i in range(len(bl_windows))]:
    line.set_zorder(10)

for line in [axes_emg_1.lines[i*N_plots+4] for i in range(len(bl_windows))]:
    line.set_zorder(10)

# Moving baseline - percentiles
bl_percentiles = [10, 20, 33, 40, 50]
colors = ['tab:purple', 'tab:cyan', 'tab:green', 'tab:red', 'tab:orange']
for _, (bl_percentile, color) in enumerate(zip(bl_percentiles, colors)):
    emg_timeseries.baseline(
        percentile=bl_percentile,
        window_s=int(7.5 * fs_emg))
    emg_di.detect_emg_breaths(
        peak_set_name='breaths',
        end_idx = int(x_lim * fs_emg),
        prominence_factor=1.0)
    emg_di.peaks['breaths'].detect_on_offset(
        baseline=emg_di.y_baseline
    )
    emg_timeseries.plot_full(
        channel_idxs=[1],
        axes=axes_emg_2,
        signal_type='env',
        baseline_bool=True,
        colors=['tab:blue', color])
    emg_di.plot_markers(peak_set_name='breaths', axes=axes_emg_2, colors=color)
axes_emg_2.set_xlabel('t (s)')
axes_emg_2.set_ylim([0.5, y_lim])
axes_emg_2.set_title('Moving baseline - Percentiles')
axes_emg_2.legend(
    [axes_emg_2.lines[i*N_plots+1] for i in range(len(bl_percentiles))],
    [str(bl_percentile) for bl_percentile in bl_percentiles],
    bbox_to_anchor=(0.01, 0.90, 0.98, 0.90),
    loc='lower left',
    ncols=5, 
    mode="expand", borderaxespad=0.)

# Ventilator data data
vent_timeseries.plot_full(axes_vent)
axis[-1].set_xlabel('t (s)')

for line in [axes_emg_2.lines[i*N_plots] for i in range(len(bl_percentiles))]:
    line.set_zorder(0)

for line in [axes_emg_2.lines[i*N_plots+1] for i in range(len(bl_percentiles))]:
    line.set_zorder(5)

for line in [axes_emg_2.lines[i*N_plots+2] for i in range(len(bl_percentiles))]:
    line.set_zorder(10)

for line in [axes_emg_2.lines[i*N_plots+3] for i in range(len(bl_percentiles))]:
    line.set_zorder(10)

for line in [axes_emg_2.lines[i*N_plots+4] for i in range(len(bl_percentiles))]:
    line.set_zorder(10)
# axis[-1, 0].axis('off')

axes_vent[-1].set_xlim([0, x_lim])

In [22]:
# Calculate the baseline for the EMG envelopes and p_vent
emg_timeseries.baseline()
emg_di.detect_emg_breaths(
        peak_set_name='breaths',
        end_idx = int(x_lim * fs_emg),
        prominence_factor=1.0)
emg_di.peaks['breaths'].detect_on_offset(
    baseline=emg_di.y_baseline
)

In [23]:
# Find supported breath pressures
vent_timeseries.baseline(
    channel_idxs=[0],
    signal_type='raw',
    percentile=50)
vent_timeseries.baseline(
    channel_idxs=[2],
    signal_type='raw',
    percentile=33)

p_vent = vent_timeseries.channels[vent_timeseries.p_vent_idx]
v_vent = vent_timeseries.channels[vent_timeseries.v_vent_idx]
vent_timeseries.find_tidal_volume_peaks(
    start_idx=0,
    end_idx=int(x_lim*fs_emg),
)
p_vent.peaks['ventilator_breaths'].detect_on_offset(
    baseline=p_vent.y_baseline)
v_vent.peaks['ventilator_breaths'].detect_on_offset(
    baseline=v_vent.y_baseline)

In [24]:
# Find the EMG peaks with the peak timing closest to the Pocc timings, and
# link ventilator Pocc peaks to EMG breaths
t_breaths_peaks_vent = p_vent.peaks['ventilator_breaths'].peak_df['peak_idx'].to_numpy() / p_vent.fs

emg_di.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_breaths_peaks_vent,
    linked_peak_set_name='breaths_linked',
)

In [ ]:
# Plot the raw data with the envelope
# EMG data
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(8, 6), sharex=True)
axes_emg = axis[:2, 0]
emg_timeseries.plot_full(axes=axes_emg, signal_type='env')
emg_di.plot_markers(peak_set_name='breaths', axes=axes_emg[1])

axes_emg[0].set_title('EMG data')
axes_emg[-1].set_xlabel('t (s)')

# Ventilator data data
axes_vent = axis[:, 1]
vent_timeseries.plot_full(axes_vent)
p_vent.plot_markers(peak_set_name='ventilator_breaths', axes=axes_vent[0])
v_vent.plot_markers(peak_set_name='ventilator_breaths', axes=axes_vent[2])
axes_vent[0].set_title('Ventilator data')
axes_vent[-1].set_xlabel('t (s)')
axes_vent[-1].set_xlim([0, x_lim])

In [ ]:
# Calculate ETPdi
emg_di.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')

print(emg_di.peaks['breaths'].peak_df)

In [ ]:
# Plot the individual peaks
n_peaks = len(emg_di.peaks['breaths'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(8, 6), sharey='row')

axes_emg = axis[0, :]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_di.plot_peaks(axes=axes_emg, peak_set_name='breaths')
    
emg_di.plot_markers(axes=axes_emg, peak_set_name='breaths')
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[1, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='ventilator_breaths')
p_vent.plot_markers(axes=axes_vent, peak_set_name='ventilator_breaths')

for axis in axes_vent:
    axis.set_xlabel('t (s)')

# 6. Test Pocc and sEMG quality of the peaks

In [ ]:
# Test EMG quality
parameter_names = {
    'time_product': 'ETPdi'
}
emg_di.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)

# The peak-validity is updated in the peak_df:
print(emg_di.peaks['breaths'].quality_values_df)
print(emg_di.peaks['breaths'].quality_outcomes_df)

In [ ]:
# Plot the individual peaks bell-fit
n_peaks = len(emg_di.peaks['breaths'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(8, 6), sharey='row')

axes_emg = axis[1, :]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_di.plot_peaks(axes=axes_emg, peak_set_name='breaths')
emg_di.plot_markers(axes=axes_emg, peak_set_name='breaths')
emg_di.plot_curve_fits(axes=axes_emg, peak_set_name='breaths')
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[0, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='ventilator_breaths')
p_vent.plot_markers(axes=axes_vent, peak_set_name='ventilator_breaths')

for axis in axes_vent:
    axis.set_xlabel('t (s)')

In [ ]:
# Plot the individual peaks bell-fit
n_peaks = len(emg_di.peaks['breaths'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(8, 6), sharey='row')

axes_emg = axis[1, :]
emg_di.plot_aub(axes=axes_emg, signal_type='env', peak_set_name='breaths')
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_di.plot_peaks(axes=axes_emg, peak_set_name='breaths')
emg_di.plot_markers(axes=axes_emg, peak_set_name='breaths')
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[0, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='ventilator_breaths')
p_vent.plot_markers(axes=axes_vent, peak_set_name='ventilator_breaths')

for axis in axes_vent:
    axis.set_xlabel('t (s)')